In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import openpyxl

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from joblib import dump, load
from openpyxl import load_workbook

# modele
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.inspection import permutation_importance

from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

### Read data

In [2]:
mix_NM_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_NM_data.csv")
mix_KM_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_KM_data.csv")
mix_CC_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_CC_data.csv")

borderline_NM_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_NM_data.csv")
borderline_KM_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_KM_data.csv")
borderline_CC_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_CC_data.csv")

smote_NM_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_NM_data.csv")
smote_KM_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_KM_data.csv")
smote_CC_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_CC_data.csv")

GAN_NM_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_NM_data.csv")
GAN_KM_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_KM_data.csv")
GAN_CC_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_CC_data.csv")

### Dictionary


In [3]:
data = {}

data["mix_NM_data"] = mix_NM_data
data["mix_KM_data"] = mix_KM_data
data["mix_CC_data"] = mix_CC_data
data["borderline_NM_data"] = borderline_NM_data
data["borderline_KM_data"] = borderline_KM_data
data["borderline_CC_data"] = borderline_CC_data
data["smote_NM_data"] = smote_NM_data
data["smote_KM_data"] = smote_KM_data
data["smote_CC_data"] = smote_CC_data
data["GAN_NM_data"] = GAN_NM_data
data["GAN_KM_data"] = GAN_KM_data
data["GAN_CC_data"] = GAN_CC_data

### Distribiution

In [4]:
output_xlsx = "D:\\ml\\undersampling_data\\reports\\ssh\\distribiution\\distribiution.xlsx"
output_txt = "D:\\ml\\undersampling_data\\reports\\ssh\\distribiution\\distribiution.txt"

all_results = pd.DataFrame()

for name, data in data.items():
    
    data = pd.DataFrame(data)
    results = data["source"].value_counts(normalize=True) * 100
    results.name = name
    print(f"Data: {name}")
    print(f"Distribiution:\n{results}")
    
    with open(output_txt, "a") as f:
        f.write(f"Data: {name}\n")
        f.write(f"Distribiution:\n{results}\n\n")
    
    # Append results to the combined DataFrame
    all_results = pd.concat([all_results, results], axis=1)
    
with pd.ExcelWriter(output_xlsx, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    all_results.to_excel(writer, sheet_name='Distribution', index=True)
    
    
#compare collect and sum_data_NM and copy column source to collect, if rows are the same
""" collect = collect.merge(sum_data_NM, on=collect.columns.tolist(), how='left', indicator=True)
collect = collect.drop(columns=["_merge"])
collect = collect.drop_duplicates()
print(collect)
print(collect["source"].value_counts()) """

Data: mix_NM_data
Distribiution:
source
original            54.635762
borderline smote    26.158940
smote               16.887417
gan                  2.317881
Name: mix_NM_data, dtype: float64
Data: mix_KM_data
Distribiution:
source
centroid    100.0
Name: mix_KM_data, dtype: float64
Data: mix_CC_data
Distribiution:
source
original            50.662252
gan                 25.165563
centroid            19.205298
borderline smote     2.980132
smote                1.986755
Name: mix_CC_data, dtype: float64
Data: borderline_NM_data
Distribiution:
source
original            52.317881
borderline smote    47.682119
Name: borderline_NM_data, dtype: float64
Data: borderline_KM_data
Distribiution:
source
centroid    100.0
Name: borderline_KM_data, dtype: float64
Data: borderline_CC_data
Distribiution:
source
original            57.947020
centroid            24.834437
borderline smote    17.218543
Name: borderline_CC_data, dtype: float64
Data: smote_NM_data
Distribiution:
source
original    54.6

' collect = collect.merge(sum_data_NM, on=collect.columns.tolist(), how=\'left\', indicator=True)\ncollect = collect.drop(columns=["_merge"])\ncollect = collect.drop_duplicates()\nprint(collect)\nprint(collect["source"].value_counts()) '